In [2]:
import numpy as np
import cPickle
from collections import defaultdict
from collections import Counter
import sys, re, os
import pandas as pd
import random,pprint
from sklearn.cross_validation import StratifiedKFold
homePath = os.path.expanduser("~")
pp = pprint.PrettyPrinter(indent = 4)

In [3]:
#google News:
importedEmbeds='/Users/tgn/Downloads/GoogleNews-vectors-negative300.bin'
usingDir='/Dropbox/pathology'

In [4]:
def build_data_cv(data_folder, cv=5, clean_string=True):
    """
    Loads data and split into 10 folds.
    """
    #get each class by dir
    classDirs=[name for name in os.listdir(os.path.normpath(homePath+labeledReportsDir)) \
                if os.path.isdir(os.path.normpath(homePath+labeledReportsDir+'/'+name))]
    classDirs.sort()
 
    print classDirs
    #build full filepaths for each labeled text file in each class dir
    fileByClass=[]
    fileByClass=[os.listdir(os.path.normpath(homePath+labeledReportsDir+'/'+thisClass)) for thisClass in classDirs]
    revs = []
    vocab = defaultdict(float)
    classes=[]
    lenCompare=[]
    lenDict={}
    for i in range(len(classDirs)):
        #for each report, we clean/tokenize, rejoin (because he did), and
        for report in fileByClass[i]:
            #build datum out of .txt file
            if report.endswith('.txt'):
                thisReportDir = os.path.normpath(homePath+labeledReportsDir+'/'+classDirs[i])
                #clean/tokenize
                cleanedReport = cleanReadReport(report,thisReportDir)
                #print(cleanedReport)
                #count vocab
                for word in cleanedReport:
                    vocab[word] += 1
                datum = {   "y": i, #classDirs[i] to get what class.
                            "text" : " ".join(cleanedReport).strip(),
                            "num_words" : len(cleanedReport)
                            #"split": np.random.randint(0,cv)
                            }
                if datum['y'] not in classes: classes.append(datum['y'])
                #print datum["text"]
                try: lenDict[len(cleanedReport)]+=1
                except KeyError: lenDict.update({len(cleanedReport):1})
                lenCompare.append([len(cleanedReport),report,classDirs[i]])
                revs.append(datum)
    orderedLens=sorted(lenDict.keys())
    print 'report len counts'
    pp.pprint([[x,lenDict[x]] for x in orderedLens])
    lenCompare=sorted(lenCompare, key=lambda x:x[0])
    print 'reports listed by len'
    for i in range(len(lenCompare)):
        lenCompare[i].append(float(i)/float(len(lenCompare)))
    pp.pprint(lenCompare)
    skf = StratifiedKFold([datum['y'] for datum in revs],n_folds=cv,shuffle=True)
 
    thisCV = 0
    for train_index, test_index in skf:
        print thisCV, [revs[datum]['y'] for datum in test_index]
        print 'counts:',Counter([revs[datum]['y'] for datum in test_index])
        for x in test_index:
            revs[x].update({'split':thisCV})
        thisCV+=1
    '''
    pos_file = data_folder[0]
    neg_file = data_folder[1]
 
    with open(pos_file, "rb") as f:
        for line in f:
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(" ".join(rev))
            else:
                orig_rev = " ".join(rev).lower()
            words = set(orig_rev.split())
            for word in words:
                voIcab[word] += 1
            datum  = {"y":1,
                      "text": orig_rev,
                      "num_words": len(orig_rev.split()),
                      "split": np.random.randint(0,cv)}
            revs.append(datum)
    with open(neg_file, "rb") as f:
        for line in f:
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(" ".join(rev))
            else:
                orig_rev = " ".join(rev).lower()
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            datum  = {"y":0,
                      "text": orig_rev,
                      "num_words": len(orig_rev.split()),
                      "split": np.random.randint(0,cv)}
            revs.append(datum)
    '''
    random.shuffle(revs)
    return revs, vocab, classDirs

In [5]:
def get_W(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size+1, k), dtype='float32')
    W[0] = np.zeros(k, dtype='float32')
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map


In [ ]:
def get_W2(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size+1, k), dtype='float32')
    W[0] = np.zeros(k, dtype='float32')
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map



In [6]:
def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, "rb") as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in xrange(vocab_size):
            word = []
            while True:
                ch = f.read(1)
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            if word in vocab:
               word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return word_vecs

In [7]:
def add_unknown_words(word_vecs, vocab, min_df=2, k=300):
    """
    For words that occur in at least min_df documents, create a separate word vector.
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            word_vecs[word] = np.random.uniform(-0.25,0.25,k)


In [8]:
#Kim's clean_str function
def clean_str(string, TREC=False):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip() if TREC else string.strip().lower()

In [9]:
def clean_str_sst(string):
    """
    Tokenization/string cleaning for the SST dataset
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [10]:
def cleanReadReport(infile,fileDir):
    #stage 1: clean document by line
    reportPath=os.path.normpath(fileDir+'/'+infile)
    outfile=""
    with open(os.path.normpath(reportPath), 'r') as inFile:
        readFile = inFile.read()
    cleanedFile = cleanDoc(readFile)
    return cleanedFile
    #stage 2: clean docment by word

In [11]:
def cleanDoc(inDoc):
    outDoc=[]
    #print '==========================================='
    #print 'inline ', inDoc
    #print '==========================================='
    for line in inDoc.splitlines():
        outDoc.append(cleanLine(line.strip()))
        output=' '.join(outDoc)
        #print 'before', output
        tokenized=[cleanWord(word) for word in output.split(" ") \
            if len(cleanWord(word))>0]
		#print 'after', tokenized
    #thisText=Text(' '.join(tokenized))
    #print '-------------------------------------------'
    #print 'outline'
    #pp.pprint(thisText.sentences)
    #print '-------------------------------------------'
    return tokenized

In [12]:
reWord=re.compile(r'[^a-zA-Z0-9]')
#sub7=re.compile(r'[0-9]{1,}[\.]{0,1}[0-9]{0,1}')
def cleanWord(text):
    text = text.strip().lower()
    #text= sub7.sub('NUMBER',text)
    return text

sub1=re.compile(r'<.*?>')
sub2=re.compile(r'\\{1,2}X[a-zA-Z0-9]{1,}\\?')
#sub3=re.compile(r'[**]\/?[^\]]*]')
sub4=re.compile(r'([^0-9\s])\1{2,}')
#sub5=re.compile(r'[[]\/?[^>]*[]]')
sub6=re.compile(r'[A-Z][a-z]+[A-Z][a-z]+[:]')


In [13]:
def cleanLine(text):
    text = sub1.sub(' ',text)
    text = sub2.sub(' ',text)
    #text = sub3.sub('',text)
    text = sub4.sub(' ',text)
    #text = sub5.sub('',text)
    text = sub6.sub(' ',text)
    text= reWord.sub(' ',text)
    return text

In [39]:
def main(someDir=usingDir):
    #get w2v pretrained
    global labeledReportsDir
    labeledReportsDir=someDir


    w2v_file = importedEmbeds
    data_folder = ["rt-polarity.pos","rt-polarity.neg"]
    print "loading data...",

    revs, vocab, importedClasses = build_data_cv(labeledReportsDir, cv=10, clean_string=True)
    max_l = np.max(pd.DataFrame(revs)["num_words"])
    print "data loaded!"
    print "number of sentences/docs: " + str(len(revs))
    print "vocab size: " + str(len(vocab))
    print "max sentence length: " + str(max_l)
    print "loading word2vec vectors...",
    w2v = load_bin_vec(w2v_file, vocab)
    print "word2vec loaded!"
    print "num words already in word2vec: " + str(len(w2v))
    add_unknown_words(w2v, vocab)
    W, word_idx_mapW2V = get_W(w2v)
#     print W
#     print word_idx_mapW2V
    rand_vecs = {}
    add_unknown_words(rand_vecs, vocab)
    print '-----------------------------'
    print rand_vecs['cancer']
    print '-----------------------------'
#     print vocab
    W2, word_idx_mapRand = get_W(vocab)
#     print W2
#     if os.path.exists('mr12index_2.g'):
#         os.remove('mr12index_2.g')
#     cPickle.dump([revs, W, W2, word_idx_mapW2V,word_idx_mapRand, vocab,max_l,importedClasses], open("mr12index_2.g", "wb"))
#     print "dataset created!"
    print vocab


In [40]:
if __name__=="__main__":
    main()

loading data... ['c34.0', 'c34.1', 'c34.2', 'c34.3', 'c34.9', 'c50.1', 'c50.2', 'c50.3', 'c50.4', 'c50.5', 'c50.8', 'c50.9']
report len counts
[   [14, 1],
    [51, 1],
    [53, 2],
    [54, 1],
    [60, 1],
    [61, 2],
    [64, 1],
    [65, 2],
    [66, 1],
    [68, 1],
    [69, 1],
    [71, 2],
    [74, 3],
    [75, 2],
    [76, 2],
    [77, 2],
    [79, 3],
    [80, 1],
    [81, 2],
    [82, 2],
    [84, 1],
    [87, 2],
    [88, 1],
    [89, 2],
    [90, 3],
    [91, 3],
    [92, 1],
    [93, 1],
    [94, 2],
    [95, 2],
    [96, 4],
    [97, 2],
    [99, 2],
    [100, 2],
    [101, 1],
    [102, 1],
    [103, 3],
    [104, 2],
    [105, 4],
    [106, 2],
    [107, 2],
    [108, 1],
    [110, 2],
    [111, 1],
    [112, 4],
    [113, 2],
    [114, 1],
    [115, 2],
    [118, 1],
    [120, 2],
    [121, 2],
    [122, 1],
    [124, 3],
    [126, 2],
    [127, 3],
    [129, 1],
    [130, 1],
    [131, 4],
    [132, 2],
    [134, 1],
    [136, 1],
    [137, 2],
    [138, 1],
    [139

NameError: name 'vocab' is not defined